In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
from konlpy.tag import Okt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
import pickle
import re
import os
import json

In [5]:
from tensorflow.keras.models import load_model

In [6]:
from tqdm import tqdm

In [15]:
okt = Okt()

In [7]:
save_dir = 'saved_models'

In [8]:
# 모델 로드
loaded_model = load_model(os.path.join(save_dir, 'best_model.keras'))

# 토크나이저 로드
with open(os.path.join(save_dir, 'tokenizer.pkl'), 'rb') as handle:
    tokenizer = pickle.load(handle)

# 학습 정보 로드
with open(os.path.join(save_dir, 'history.pkl'), 'rb') as handle:
    loaded_history = pickle.load(handle)

# 데이터 전처리 정보 로드
with open(os.path.join(save_dir, 'preprocessing_info.json'), 'r') as json_file:
    loaded_preprocessing_info = json.load(json_file)

In [ ]:
loaded_preprocessing_info

In [11]:
max_len = loaded_preprocessing_info['max_len']

In [9]:
# 삼성전자 하루치 샘플 데이터
df = pd.read_csv('practice_Toss_KOSPI_2025-01-18.csv')

In [ ]:
# 실제 샘플 테스트

def sentiment_predict(new_sentence):
  
  # URL 패턴 확인
  # http:// 또는 https://로 시작하는 URL, www.로 시작하는 문자열, .com 또는 .net으로 끝나는 문자열 => 광고성 추측
  if re.search(r'http[s]?://|www\.|\.com|\.net', new_sentence):
    return None, 'advertisement'  # 광고성 내용 처리
    
  # 특수 문자와 공백 체크
  if re.match(r'^[^ㄱ-ㅎㅏ-ㅣ가-힣]+$', new_sentence.strip()):
    return None, 'empty'  # 빈 문장 처리  
    
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', new_sentence) # 특수 문자 제거
  
  if not new_sentence.strip():  # 특수 문자 제거 후 빈 문자열인지 확인
    return None, 'empty'  # 빈 문장 처리
  
  new_sentence = okt.morphs(new_sentence, stem=True)             # 토큰화
  encoded = tokenizer.texts_to_sequences([new_sentence])         # 정수 인코딩 
  pad_new = pad_sequences(encoded, maxlen=max_len)               # 패딩 
  predicted_value = loaded_model.predict(pad_new)                # 예측
  
  score = float(predicted_value[0])
  return score, None  

# 예측 결과를 저장할 리스트
results = []

# 각 샘플 문장에 대해 예측 수행
for idx, sentence in enumerate(df['내용'].values):
  
  # 예측 수행
  score, tag = sentiment_predict(sentence)

  if tag == 'advertisement':
      results.append({
          '문장': sentence,
          '예측 확률': None,
          '예측 레이블': 'advertisement'
      })
      continue
  elif tag == 'empty':
      results.append({
          '문장': sentence,
          '예측 확률': None,
          '예측 레이블': 'empty'
      })
      continue

  if score is not None:
      score_percentage = score * 100 if score > 0.5 else (1 - score) * 100
      results.append({
          '문장': sentence,
          '예측 확률': score_percentage,
          '예측 레이블': 1 if score > 0.5 else 0,
      })
    
# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 확률을 소수점 두 자리로 포맷하여 백분율로 변환
results_df['예측 확률'] = results_df['예측 확률']\
                      .apply(lambda x: "{:.2f}%".format(x) if x is not None else 'N/A')
                      
# 예측 라벨링 기반 긍정점수 계산
# 라벨이 1인 레이블의 비율 계산
total_count = len(results_df[results_df['예측 레이블'].isin([0, 1])])  # 0과 1 레이블의 수
positive_count = len(results_df[results_df['예측 레이블'] == 1])  # 라벨이 1인 경우의 수

if total_count > 0:
    positive_ratio = (positive_count / total_count) * 100
else:
    positive_ratio = 0

display(results_df)
print(f'이 종목의 긍정 점수 : {positive_ratio:.2f}% ')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


C:\Users\hydrl\AppData\Local\Temp\ipykernel_10348\3839959439.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  score = float(predicted_value[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━

,문장,예측 확률,예측 레이블
0,배당금 받은걸로 조금씩 추매하고..,99.65%,1
1,https://m.site.naver.com/1AS4j,nan%,advertisement
2,망햇다,53.36%,0
3,민노총에게 구타당하고.\n보수집회에는 시비거는 이상한 경찰.\n세금을 민노총ㆍ중공에...,nan%,advertisement
4,아 묶였네 언제 오르나,71.71%,0
...,...,...,...
575,오이한번만 오이주이소,80.50%,1
576,삼전평단\n62000\n삼물평단170000\n상성그룹경영진의소액주주홀대피해를당하면서...,99.32%,0
577,삼성 액시노스 거부한 TSMC https://kekewo.net/tsmc-%ec%8...,nan%,advertisement
578,곧 봄이다 내 삼전주에도 빨간 물이들길..,88.77%,0


이 종목의 긍정 점수 : 35.01% 
